## Analyse Exploratoire

### Distribution
![Distribution](imgs/Distribution.png)

In [64]:
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

matplotlib.use("Qt5Agg")

# Charger le CSV
df = pd.read_csv("data/fichier-de-donnees-mixtes.csv")

# Description du fichier
print(f"Description : {df.describe}")

# Encodage du sexe, smoker, sport_licence en entier 0 ou 1
encoder = LabelEncoder()
df["sexe"] = encoder.fit_transform(df["sexe"])
df["smoker"] = encoder.fit_transform(df["smoker"])
df["sport_licence"] = encoder.fit_transform(df["sport_licence"])
df["nationalité_francaise"] = encoder.fit_transform(df["nationalité_francaise"])

# Catégoriel
mask = df["situation_familiale"].notna()
df.loc[mask, "situation_familiale"] = encoder.fit_transform(df.loc[mask, "situation_familiale"])
df["region"] = encoder.fit_transform(df["region"])

# Ordinal
ordre = [["aucun", "bac", "bac+2", "master", "doctorat"]]
ordinal_enc = OrdinalEncoder(categories=ordre)
df["niveau_etude"] = ordinal_enc.fit_transform(df[["niveau_etude"]])

# Extraction de l'année
df["date_creation_compte"] = pd.to_datetime(df["date_creation_compte"], errors="coerce")
df["date_creation_compte"] = df["date_creation_compte"].dt.year
df.rename(columns={"date_creation_compte": "annee_creation_compte"})

# Affichage de la distribution des données
plt.figure(figsize=(15, 12))

# Sélection des colonnes numériques uniquement
num_cols = df.select_dtypes(include="number").columns

for i, col in enumerate(num_cols, 1):
    plt.subplot(4, 5, i)     # 4x5
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f"Distribution de {col}")
    plt.xlabel(col)
    plt.ylabel("Fréquence")

plt.tight_layout()
plt.show()

#display(df.head(20))

Description : <bound method NDFrame.describe of             nom   prenom  age  taille  poids sexe sport_licence niveau_etude  \
0       Ramirez    Casey   73   161.1   67.3    H           non       master   
1          Hill  Phillip   44   168.2   74.9    H           non          bac   
2     Hernandez   Martin   71   160.3   45.5    H           non     doctorat   
3        Miller  Michael   62   161.9   87.7    F           oui          bac   
4        Walker  Matthew   18   178.0   77.6    F           oui          bac   
...         ...      ...  ...     ...    ...  ...           ...          ...   
9995      Ortiz  Chelsea   37   163.6   96.7    F           non          bac   
9996   Gonzalez   Edward   19   157.2   55.3    F           non          bac   
9997     Potter  Allison   71   163.0   77.2    H           non        aucun   
9998    Bonilla   Alyssa   54   165.7   73.2    H           oui        aucun   
9999      Payne   Kristy   54   173.9   77.8    H           non       ma

## Heatmap

![Correlation](imgs/Correlation.png)

In [61]:
## Heatmap de corrélation
plt.figure(figsize=(10, 8))
corr = df.corr(numeric_only=True)

sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matrice de corrélation")
plt.show()

## Identifier les valeurs manquantes

Utilisation de missingno

![Colonnes manquantes](imgs/Colonnes.png)
![Colonnes manquantes 2](imgs/Colonnes2.png)

In [65]:
# Visualiser la matrice des valeurs manquantes
msno.matrix(df)
plt.show()

# Visualiser le barplot des colonnes manquantes
msno.bar(df)
plt.show()

# Nombre de lignes avec au moins une valeur manquante
missing_line = df.isna().any(axis=1).sum()
total_line = df.shape[0]
print(f"Nb lignes manquantes : {missing_line} / {total_line}")

Nb lignes manquantes : 8802 / 10000


## Nettoyage des Données

### Identifier les colonnes utiles au résultat




#### Supprimer les colonnes non pertinentes

4 colonnes sont partielles, les autres sont complètes. Je fais le choix d'éliminer les colonnes avec beaucoup de données manquantes (> 40%).
**historique_credit** et **score_credit** seront supprimées.
Il restera **situation_familliale** et **loyer_mensuel**.
**situation_familliale** n'est pas imputable, je la retire donc manuellement.
Et pour **loyer_mensuel** je remplace les valeurs manquantes par la médiane.

In [66]:
cols_before = df.shape[1]
threshold = 0.40  # 40%
a_supprimer = list(df.columns[df.isna().mean() > threshold]) + ["situation_familiale"]
df_clean = df.drop(columns=a_supprimer)
cols_after = df_clean.shape[1]
print(f"Nb colonnes avant nettoyage : {cols_before}, nb colonnes après : {cols_after}")


Nb colonnes avant nettoyage : 19, nb colonnes après : 16


#### Imputation
On remplit les valeurs manquantes de **loyer_mensuel** avec la valeur médiane.

In [67]:
nans_before = df_clean["loyer_mensuel"].isna().sum()
df_clean["loyer_mensuel"] = df_clean["loyer_mensuel"].fillna(df_clean["loyer_mensuel"].median())
nans_after = df_clean["loyer_mensuel"].isna().sum()

Au cas où il resterait des lignes avec des champs manquants, je supprime les lignes concernées. Il n'y en a pas actuellement dans le jeu de données, mais cela pourrait être le cas dans le futur.

In [68]:
lines_before = df_clean.shape[0]
# Supprimer les lignes contenant des valeurs manquantes
df_clean = df_clean.dropna()
lines_after = df_clean.shape[0]
print(f"Nb lignes avant nettoyage : {lines_before}, nb lignes après : {lines_after}")

Nb lignes avant nettoyage : 10000, nb lignes après : 10000


### Ecarter les outliers
Filtrer les outliers avec la méthode IQR (sur toutes les colonnes)

In [71]:
lines_before = df_clean.shape[0]

# Sélection des colonnes de type continu
colonnes_outliers = ["age", "taille", "poids", "revenu_estime_mois", "montant_pret"]

# On sélectionne uniquement ces colonnes (numeric obligatoire)
df_num = df_clean[colonnes_outliers].select_dtypes(include="number")

# Calcul IQR
Q1 = df_num.quantile(0.25)
Q3 = df_num.quantile(0.75)
IQR = Q3 - Q1

# Détection des outliers uniquement sur ces colonnes
mask_outliers = ((df_num < (Q1 - 1.5 * IQR)) | (df_num > (Q3 + 1.5 * IQR))).any(axis=1)

# Suppression des lignes outliers dans le DF complet
df_filtered = df_clean[~mask_outliers].copy()

lines_after = df_filtered.shape[0]
print(f"Nb lignes avant filtrage des outliers : {lines_before}, nb lignes après : {lines_after}")


Nb lignes avant filtrage des outliers : 10000, nb lignes après : 9751


#### Standardisation des données
La standardisation consiste à transformer chaque variable pour qu’elle ait une moyenne de 0 et un écart-type de 1.  
Cela permet de mettre toutes les colonnes numériques sur la même échelle, ce qui est essentiel pour de nombreux algorithmes de Machine Learning.




In [72]:
from sklearn.preprocessing import StandardScaler

# Initialiser le scaler
scaler = StandardScaler()

# Colonnes numériques
num_cols = df_filtered.select_dtypes(include="number").columns

# Copie du dataset
df_standardized = df_filtered.copy()

# Application de la standardisation
df_standardized[num_cols] = scaler.fit_transform(df_filtered[num_cols])

print("Standardisation effectuée (aperçu) :")
display(df_standardized.head())

# Sauvegarde du dataset standardisé
output_path = "data/dataset_standardized.csv"
df_standardized.to_csv(output_path, index=False)

print(f"Dataset sauvegardé sous : {output_path}")



Standardisation effectuée (aperçu) :


,nom,prenom,age,taille,poids,sexe,sport_licence,niveau_etude,region,smoker,nationalité_francaise,revenu_estime_mois,risque_personnel,date_creation_compte,loyer_mensuel,montant_pret
0,Ramirez,Casey,1.571914,-0.910330,-0.192590,1.001129,-0.969195,0.722076,0.646162,-1.008342,0.815904,-1.447571,-1.338992,1.109858,-1.186436,0.429579
1,Hill,Phillip,-0.151545,-0.184470,0.327662,1.001129,-0.969195,-0.689363,1.519263,-1.008342,0.815904,2.425334,1.002711,0.479817,1.542259,2.312510
2,Hernandez,Martin,1.453054,-0.992117,-1.684891,1.001129,-0.969195,1.427795,-1.536589,0.991727,0.815904,1.142898,-1.270118,0.479817,-0.040138,0.900855
3,Miller,Michael,0.918188,-0.828543,1.203875,-0.998873,1.031784,-0.689363,-1.536589,0.991727,0.815904,0.700709,-0.615819,1.109858,1.542259,0.708102
4,Walker,Matthew,-1.696715,0.817420,0.512488,-0.998873,1.031784,-0.689363,-0.663488,-1.008342,0.815904,1.232041,0.555033,0.479817,-0.040138,0.163597


Dataset sauvegardé sous : data/dataset_standardized.csv


In [73]:
## Heatmap de corrélation sur les données nettoyées et standardisées
plt.figure(figsize=(10, 8))
corr = df_standardized.corr(numeric_only=True)

sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matrice de corrélation")
plt.show()

### Analyse
D'après la matrice, nous constatons que le sexe et la nationnalité sont en corrélation avec l'estimation du prêt. Le jeu de données est donc biaisé. Garder ces données provoquerait de la discrimination.

Dans la version éthique du jeu de données, je supprime les colonnes suivantes :
- nom et prénom : ce sont des données d'identification directe
- nationnalité et sexe qui dont des données sensibles interdites par la RGPD
- smoker qui est une données de santé et potentiellement discriminatoire
- situation_familiale qui est une données personnelle et provoquerait de la discrimination entre personne seules ou en couple (à de toute façon était supprimé en amont pour manque de données)
- risque_personnel : variable non neutre donc à sortir d'un jeu de données éthique, même si dans zone grise de la RGPD

In [76]:
# Création de la version éthique : suppression des variables sensibles
colonnes_sensibles = [
    "nom", "prenom", "sexe", "smoker", "nationalité_francaise", "risque_personnel"
]

df_ethic = df_standardized.drop(columns=colonnes_sensibles)

output_ethic = "data/dataset_ethic.csv"
df_ethic.to_csv(output_ethic, index=False)

print(f"Dataset éthique sauvegardé sous : {output_ethic}")

KeyError: "['situation_familiale'] not found in axis"